In [ ]:
! pip install -q fasttext
! pip install -q ydata-profiling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import fasttext
import re
import spacy

from sklearn.manifold import TSNE

import tensorflow as tf
from tensorflow.keras.datasets import imdb

import warnings
warnings.filterwarnings('ignore')
from IPython.display import display , HTML


In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value , key) for (key , value) in word_index.items()])

def decode_review(text):
    return " ".join([reverse_word_index.get(i-3 , "?") for i in text])

(x_train , y_train) , (x_test , y_test) = imdb.load_data(num_words = 10000)

x_train = np.array([decode_review(x) for x in x_train[:4000]])
y_train = np.array(y_train[:4000])

x_test = np.array([decode_review(x) for x in x_test[:4000]])
y_test = np.array(y_test[:4000])

df_train = pd.DataFrame({"review" : x_train , "sentiment" : y_train})
df_test = pd.DataFrame({"review" : x_test , "sentiment" : y_test})
df = pd.concat([df_train , df_test])

In [ ]:
df

,review,sentiment
0,? this film was just brilliant casting locatio...,1
1,? big hair big boobs bad music and a giant saf...,0
2,? this has to be one of the worst films of the...,0
3,? the ? ? at storytelling the traditional sort...,1
4,? worst mistake of my life br br i picked this...,0
...,...,...
3995,? first of all around the time i wrote this co...,1
3996,? you don't have to be a to appreciate this ge...,1
3997,? this movie deals with the european ? exchang...,1
3998,? you could get into the ? gritty of this film...,1


In [ ]:
df.shape

(8000, 2)

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df , title="Profiling Report", explorative=True)
profile.to_file("profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
HTML(filename='profiling_report.html')

In [ ]:
df = df.drop_duplicates()

In [ ]:
nlp = spacy.load("en_core_web_sm" , disable=["parser", "ner", "lemmatizer"])

def preprocess_text(text):
    doc = nlp(str(text).lower())
    tokens = [token.text for token in doc if token.is_alpha or token.is_digit and not token.is_stop]
    return " ".join(tokens)

df['review'] = df['review'].apply(preprocess_text)


In [ ]:
import random

i = random.randint(0 , len(df) - 1)
print(f"\nBefore: {df["review"].iloc[i][:80]}")
print(f"\nAfter: {df["review"].iloc[i][:80]}")


Before: with batman returns tim burton to an important in american cinema giving a seque

After: with batman returns tim burton to an important in american cinema giving a seque


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(df["review"] , df["sentiment"].values , test_size=0.2 , random_state=42 , stratify=df["sentiment"].values)
print(f"Shape of training data: {x_train.shape}")
print(f"Shape of testing data: {x_test.shape}")

Shape of training data: (6391,)
Shape of testing data: (1598,)


In [ ]:
def to_fasttext_format(texts, labels, path):
    with open(path, "w", encoding="utf-8") as f:
        for text, label in zip(texts, labels):
            f.write(f"__label__{label} {text}\n")

to_fasttext_format(x_train, y_train, "train_fasttext.txt")


In [ ]:
ft_model = fasttext.train_supervised(
    input="train_fasttext.txt",
    epoch=25,
    lr=0.5,
    wordNgrams=2,
    dim=300,
    verbose=0
)

In [ ]:
x_train_emb = np.vstack([ft_model.get_sentence_vector(text) for text in x_train])
x_test_emb = np.vstack([ft_model.get_sentence_vector(text) for text in x_test])

In [ ]:
print(x_train_emb.shape)
print(x_test_emb.shape)


(6391, 300)
(1598, 300)


In [ ]:
from lightgbm import LGBMClassifier

print("Training LightGBM model...")

lgbm_model = LGBMClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=-1
)

lgbm_model.fit(x_train_emb, y_train, eval_set=[(x_test_emb, y_test)], eval_metric="logloss")

print("✓ Model trained")

Training LightGBM model...
✓ Model trained


In [ ]:
y_pred_lgbm = lgbm_model.predict(x_test_emb)

In [ ]:
# Classification report
print(classification_report(y_test , y_pred_lgbm))

              precision    recall  f1-score   support

           0       0.89      0.86      0.87       805
           1       0.86      0.89      0.88       793

    accuracy                           0.87      1598
   macro avg       0.88      0.87      0.87      1598
weighted avg       0.88      0.87      0.87      1598

